# Import packages

In [18]:
%matplotlib qt5

import matplotlib.pyplot as mplt
mplt.rcParams.update({
    'lines.linewidth': 2, 'axes.grid': True,
    'grid.alpha': 0.5, 'grid.linewidth': 1, 'grid.linestyle': '--',
    'font.size': 18
})

import numpy as np

from siriuspy.clientconfigdb import ConfigDBClient
from siriuspy.devices import HLFOFB, SOFB
from fofb_sofb_interaction import FOFB_SOFB

# Create Objects

In [6]:
# fofb = HLFOFB()
# sofb = SOFB(SOFB.DEVICES.SI)

In [9]:
# mat_s = sofb.respmat
# mat_f = fofb.respmat

In [19]:
clt = ConfigDBClient()

mat_s = np.array(clt.get_config_value(
    'ref_respmat', config_type='si_orbcorr_respm'))
mat_f = np.array(clt.get_config_value(
    'ref_respmat', config_type='si_fastorbcorr_respm'))

# Configure objects and perform simulation

In [20]:
mat_s_rand = np.random.rand(*mat_s.shape)
mat_f_rand = np.random.rand(*mat_f.shape)
mat_f_rand[:, -1] = mat_s_rand[:, -1]

In [21]:
fofb_sofb = FOFB_SOFB(mat_s, mat_f)
# fofb_sofb = FOFB_SOFB(mat_s_rand, mat_f_rand)

In [22]:
enblx = fofb_sofb.bpmxenbl_fofb.reshape(20, -1)
enbly = fofb_sofb.bpmyenbl_fofb.reshape(20, -1)
enblx[:, [1, 2, 5, 6]] = False
enbly[:, [1, 2, 5, 6]] = False
fofb_sofb.bpmxenbl_fofb = enblx.ravel()
fofb_sofb.bpmyenbl_fofb = enbly.ravel()

# enblx = fofb_sofb.chenbl_fofb.reshape(20, -1)
# enbly = fofb_sofb.cvenbl_fofb.reshape(20, -1)
# enblx[:, [1, 2]] = False
# enbly[:, [1, 2]] = False
# fofb_sofb.chenbl_fofb = enblx.ravel()
# fofb_sofb.cvenbl_fofb = enbly.ravel()

# fofb_sofb.rfenbl_fofb = False

print(fofb_sofb)

SOFB Parameters:                 FOFB Parameters:                 Interaction Parameters:
minsv_sofb  : 0.00               minsv_fofb  : 0.00               gaini_down  : 0.50 
regc_sofb   : 0.00               regc_fofb   : 0.00               gaini_orbu  : 0.50 
gaini_sofb  : 0.50               gaini_fofb  : 2500.00            f0_fil_down : 25.00 [Hz]
gainp_sofb  : 0.00               gainp_fofb  : 0.00               f0_ac_fofb  : 1.00 [Hz]
f0_fil_sofb : 25.00 [Hz]         f0_fil_fofb : 25.00 [kHz]        ac_fofb     : False
f0_plt_sofb : 100.00 [Hz]        f0_plt_fofb : 10.00 [kHz]        proj_null_space: False
dly_fil_sofb: 30.00 [ms]         dly_fil_fofb: 200.00 [us]        zero_distortion: False
dly_plt_sofb: 10.00 [ms]         dly_plt_fofb: 50.00 [us]         

SOFB Enable Lists:                                  FOFB Enable Lists:
RF = True                                           RF = True
        BPMx      BPMy      CH       CV                     BPMx      BPMy     CH    CV   
0

## Sweep Parameters

In [32]:
fofb_sofb.gaini_fofb = 0.1 * 25e3
fofb_sofb.gaini_sofb = 0.5 * 1
fofb_sofb.ac_fofb = False
fofb_sofb.gaini_down = 0.02 * 25 * 0
fofb_sofb.gaini_orbu = 1 * 25

In [33]:
npts = 150
freqs = np.logspace(-5, 0, npts)

svals = []
vals = [0.25, 2.5, 25, 50]
names = [f'Go={v:.1f}' for v in vals]

svals = fofb_sofb.sweep_parameter(freqs, 'gaini_orbu', vals, names, only_fofb=False)


Calulating configuration: Go=0.2
SOFB Parameters:                 FOFB Parameters:                 Interaction Parameters:
minsv_sofb  : 0.00               minsv_fofb  : 0.00               gaini_down  : 0.00 
regc_sofb   : 0.00               regc_fofb   : 0.00               gaini_orbu  : 0.25 
gaini_sofb  : 0.50               gaini_fofb  : 2500.00            f0_fil_down : 25.00 [Hz]
gainp_sofb  : 0.00               gainp_fofb  : 0.00               f0_ac_fofb  : 1.00 [Hz]
f0_fil_sofb : 25.00 [Hz]         f0_fil_fofb : 25.00 [kHz]        ac_fofb     : False
f0_plt_sofb : 100.00 [Hz]        f0_plt_fofb : 10.00 [kHz]        proj_null_space: False
dly_fil_sofb: 30.00 [ms]         dly_fil_fofb: 200.00 [us]        zero_distortion: False
dly_plt_sofb: 10.00 [ms]         dly_plt_fofb: 50.00 [us]         
000/150 -> 0.000 Hz
010/150 -> 0.000 Hz
020/150 -> 0.000 Hz
030/150 -> 0.000 Hz
040/150 -> 0.000 Hz
050/150 -> 0.000 Hz
060/150 -> 0.001 Hz
070/150 -> 0.002 Hz
080/150 -> 0.005 Hz
090/150 -> 0

In [31]:
svs = slice(None, None, 20)
fig, axs = fofb_sofb.plot_singular_values(freqs, svals, names, svs, hide_propties=['gaini_orbu'])

## Compare Loop states

In [24]:
fofb_sofb.gaini_fofb = 0.1 * 25e3
fofb_sofb.gaini_sofb = 0.5 * 1
fofb_sofb.ac_fofb = True
fofb_sofb.gaini_down = 0.02 * 25 * 1
fofb_sofb.gaini_orbu = 1 * 25

In [36]:
npts = 300
freqs = np.logspace(-4, 2, npts)
names = ['F', 'S', 'F+S', 'F+S+D', 'F+S+O', 'F+S+D+O']
states = [0b1, 0b100, 0b101, 0b1101, 0b10101, 0b11101]
# states = states[:2]
# names = names[:2]
variables = ['gaini_fofb', 'ac_fofb', 'gaini_sofb', 'gaini_down', 'gaini_orbu']

svals = fofb_sofb.compare_loop_states(freqs, variables, states, names, only_fofb=False)


Calculating configuration: F
SOFB Parameters:                 FOFB Parameters:                 Interaction Parameters:
minsv_sofb  : 0.00               minsv_fofb  : 0.00               gaini_down  : 0.00 
regc_sofb   : 0.00               regc_fofb   : 0.00               gaini_orbu  : 0.00 
gaini_sofb  : 0.00               gaini_fofb  : 2500.00            f0_fil_down : 25.00 [Hz]
gainp_sofb  : 0.00               gainp_fofb  : 0.00               f0_ac_fofb  : 1.00 [Hz]
f0_fil_sofb : 25.00 [Hz]         f0_fil_fofb : 25.00 [kHz]        ac_fofb     : False
f0_plt_sofb : 100.00 [Hz]        f0_plt_fofb : 10.00 [kHz]        proj_null_space: False
dly_fil_sofb: 30.00 [ms]         dly_fil_fofb: 200.00 [us]        zero_distortion: False
dly_plt_sofb: 10.00 [ms]         dly_plt_fofb: 50.00 [us]         
000/300 -> 0.000 Hz
010/300 -> 0.000 Hz
020/300 -> 0.000 Hz
030/300 -> 0.000 Hz
040/300 -> 0.001 Hz
050/300 -> 0.001 Hz
060/300 -> 0.002 Hz
070/300 -> 0.003 Hz
080/300 -> 0.004 Hz
090/300 -> 0.006

140/300 -> 0.064 Hz
150/300 -> 0.102 Hz
160/300 -> 0.162 Hz
170/300 -> 0.258 Hz
180/300 -> 0.409 Hz
190/300 -> 0.650 Hz
200/300 -> 1.031 Hz
210/300 -> 1.637 Hz
220/300 -> 2.598 Hz
230/300 -> 4.125 Hz
240/300 -> 6.547 Hz
250/300 -> 10.393 Hz
260/300 -> 16.496 Hz
270/300 -> 26.185 Hz
280/300 -> 41.565 Hz
290/300 -> 65.978 Hz
finished!


In [37]:
svs = slice(None, None, 20)
fig, axs = fofb_sofb.plot_singular_values(freqs, svals, names, svs, hide_propties=variables)

/home/fernando/repos/testing-scripts/fofb_sofb_interaction.py:483: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  ads.set_yscale('log')
